In [20]:
import os
import numpy as np
import matplotlib.pyplot as plt
import glob
from skimage import transform
import nibabel as nib
import tqdm
from PIL import Image


def normalize_img(img):
    # Warning for when dividing NaN value??
    norm_img = np.divide(img,np.max(img))
    return norm_img


def crop_pad_resize(image, nx, ny):
    x, y = image.shape

    # difference in nr of pixels (divide by 2 since we have 2 sides)
    x_s = (x - nx) // 2
    y_s = (y - ny) // 2
    x_c = (nx - x) // 2
    y_c = (ny - y) // 2

    if x > nx and y > ny:
        # if image is larger in both dimensions cut a slice
        slice_cropped = image[x_s:x_s + nx, y_s:y_s + ny]

    else:
        # if one dim is smaller fill that side up with 0's
        slice_cropped = np.zeros((nx, ny))

        if x <= nx and y > ny:
            # fill up x direction with 0's, cut in x direction
            slice_cropped[x_c:x_c + x, :] = image[:, y_s:y_s + ny]
        elif x > nx and y <= ny:
            # fill up y direction with 0's, cut in y direction
            slice_cropped[:, y_c:y_c + y] = image[x_s:x_s + nx, :]
        else:
            # if dimensions are as desired, keep the original slice
            slice_cropped[x_c:x_c + x, y_c:y_c + y] = image[:, :]

    return slice_cropped


def preprocess(input_folder, target_resolution, target_size):
    '''
    This function preprocesses ACDC data. It crops all images to the same size,
    transforms everything to the same resolution and normalizes the images.
    It automatically makes the folder where preprocessed data is written to,
    in the same format as the ACDC data is given. The images are in PNG-format.
    
    input_folder: the folder where raw ACDC data is located.
    target_resolution: desired resolution, should be a tuple with 2 items (x- and y-dimensions).
    target_size: desired size. Should be a tuple wiht 2 items (x- and y-dimensions).
    '''
    corrupted_files = []
    nx, ny = target_size
    data_folder = input_folder
    
    if not os.path.exists('preprocessed'):
        os.mkdir('preprocessed')
    else:
        print('Preprocessed folder already exists. Continuing regardless.')
    
    # Loop over train and test folders
    for train_test in ['training', 'testing']:

        input_folder = os.path.join(data_folder, train_test)
        len_inp = len(input_folder)+1
        
        # Make train and test folders in preprocessed folder
        if not os.path.exists(os.path.join('preprocessed/', train_test)):
            os.mkdir(os.path.join('preprocessed/', train_test))
        else:
            print('T'+train_test[1:]+' folder already exists. Continuing regardless.')
        
        # Loop over patient folders
        for folder in os.listdir(input_folder):
            
            if folder != '.ipynb_checkpoints':  # Sometimes trouble with automatically made files

                folder_path = os.path.join(input_folder, folder)
                
                # Make patient folders in preprocessed folder
                if not os.path.exists(os.path.join('preprocessed/'+train_test, folder_path[len_inp:])):
                    os.mkdir(os.path.join('preprocessed/'+train_test, folder_path[len_inp:]))
                else:
                    print('Folder for '+folder_path[len_inp:]+' already exists. Continuing regardless.')
                
                lst = os.listdir('preprocessed/'+train_test+'/'+folder_path[len_inp:])
                
                if len(lst) == 0:  # Only create files if the designated folder is empty
                    
                    for file in glob.glob(os.path.join(folder_path, 'patient???_frame??.nii.gz')):

                        # Save information about patient
                        with open(os.path.join(folder_path, 'Info.cfg')) as f:
                            lines = f.readlines()

                        ED = int(lines[0].strip()[-2:])
                        ES = int(lines[1].strip()[-2:])

                        # Split file name
                        file_base = file.split('.nii.gz')[0]
                        file_mask = file_base + '_gt.nii.gz'

                        # Load data from .nii.gz files
                        img_nii = nib.load(file)
                        img_dat = img_nii.get_fdata()

                        mask_nii = nib.load(file_mask)
                        mask_dat = mask_nii.get_fdata()

                        img = img_nii.get_fdata()
                        mask = mask_nii.get_fdata()

                        pixel_size = img_nii.header.get_zooms()

                        # Make vector to make all images have the same resolution
                        scale_vector = [pixel_size[0] / target_resolution[0], pixel_size[1] / target_resolution[1]] 

                        for zz in tqdm.tqdm(range(img.shape[2])):

                            # Normalize, rescale and crop the image and  mask

                            slice_img = np.squeeze(img[:, :, zz])
                            slice_img = normalize_img(np.squeeze(img[:, :, zz]))
                            img_rescaled = transform.rescale(slice_img,
                                                             scale_vector,
                                                             order=1,
                                                             preserve_range=True,
                                                             mode='constant')

                            slice_mask = np.squeeze(mask[:, :, zz])
                            # slice_mask = normalize_img(np.squeeze(mask[:, :, zz]))
                            mask_rescaled = transform.rescale(slice_mask,
                                                              scale_vector,
                                                              order=0,
                                                              preserve_range=True,
                                                              mode='constant')

                            img_cropped = crop_pad_resize(img_rescaled, nx, ny)
                            mask_cropped = crop_pad_resize(mask_rescaled, nx, ny)
                            
                            if(len(np.unique(mask_cropped))>4):
                                print(np.unique(mask_cropped))
                                print(file_base)
                                corrupted_files.append(file_base)

                            # Save images in PNG format
                            if 'frame{:02}'.format(ED) in file:
                                img_loc = os.path.join('preprocessed/'+train_test, file[len_inp:-7]+'_slice{:01}_ED'.format(zz)+'.png')
                                img_fin = Image.fromarray(np.uint8(255 * img_cropped),mode="L")
                                img_fin.save(img_loc, format='PNG')

                                mask_loc = os.path.join('preprocessed/'+train_test, file[len_inp:-7]+'_slice{:01}_ED_gt'.format(zz)+'.png')
                                mask_fin = Image.fromarray(np.uint8(255 * mask_cropped), mode="L")
                                mask_fin.save(mask_loc, format='PNG')
                            else:
                                img_loc = os.path.join('preprocessed/'+train_test, file[len_inp:-7]+'_slice{:01}_ES'.format(zz)+'.png')
                                img_fin = Image.fromarray(np.uint8(255 * img_cropped), mode="L")
                                img_fin.save(img_loc, format='PNG')

                                mask_loc = os.path.join('preprocessed/'+train_test, file[len_inp:-7]+'_slice{:01}_ES_gt'.format(zz)+'.png')
                                mask_fin = Image.fromarray(np.uint8(255 * mask_cropped),mode="L")
                                mask_fin.save(mask_loc, format='PNG')
                else:
                    print('Folder for '+folder_path[len_inp:]+' is not empty. No files were written to this folder.')
        
    print(corrupted_files)
    print("Preprocessing Finished")



In [21]:
target_resolution = (1.36719, 1.36719)
target_size = (212, 212)
data_path = './database'

preprocess(data_path, target_resolution, target_size)


Preprocessed folder already exists. Continuing regardless.
Training folder already exists. Continuing regardless.


100%|██████████| 8/8 [00:00<00:00, 51.29it/s]


[0.00000000e+00 5.90174868e-05 1.18034974e-04 7.51923582e-03
 7.57825331e-03 7.72872521e-03 7.78774270e-03 7.90577767e-03
 1.50384716e-02 1.51565066e-02 1.51889435e-02 1.52479610e-02
 1.53069785e-02 1.54574504e-02 1.55754854e-02 3.03778871e-02
 3.04959221e-02 3.06139570e-02 3.81656298e-02 3.82246473e-02
 9.84693021e-01 9.84752039e-01 9.84811056e-01 9.92212257e-01
 9.92271275e-01 9.92330292e-01 9.92421747e-01 9.92480764e-01
 9.99940983e-01 1.00000000e+00 1.00005902e+00 1.00751924e+00
 1.00757825e+00 1.00772873e+00 1.00778774e+00 1.01518894e+00
 1.01524796e+00 1.96950408e+00 1.96962211e+00 1.98442451e+00
 1.98454255e+00 1.98475204e+00 1.98481106e+00 1.98484349e+00
 1.98496153e+00 1.99221226e+00 1.99227127e+00 1.99248076e+00
 1.99988197e+00 1.99994098e+00 2.00000000e+00 2.00005902e+00
 2.00751924e+00 2.00757825e+00 2.00772873e+00 2.00778774e+00
 2.01518894e+00 2.01524796e+00 2.01530698e+00 2.98475204e+00
 2.98481106e+00 2.99221226e+00 2.99227127e+00 2.99242175e+00
 2.99248076e+00 2.999940

 38%|███▊      | 3/8 [00:00<00:00, 25.66it/s]

[0.00000000e+00 5.90174868e-05 1.18034974e-04 2.36069947e-04
 7.51923582e-03 7.57825331e-03 7.63727079e-03 7.72872521e-03
 7.78774270e-03 1.50384716e-02 1.51565066e-02 1.51889435e-02
 1.52479610e-02 1.54574504e-02 1.55754854e-02 3.03778871e-02
 3.04369046e-02 3.04959221e-02 9.84693021e-01 9.84752039e-01
 9.84811056e-01 9.92212257e-01 9.92271275e-01 9.92421747e-01
 9.92480764e-01 9.99940983e-01 1.00000000e+00 1.00005902e+00
 1.00751924e+00 1.00761069e+00 1.00772873e+00 1.00778774e+00
 1.01518894e+00 1.01524796e+00 1.96950408e+00 1.96962211e+00
 1.97729182e+00 1.98442451e+00 1.98454255e+00 1.98469302e+00
 1.98475204e+00 1.98481106e+00 1.98484349e+00 1.98490251e+00
 1.98496153e+00 1.98507956e+00 1.99221226e+00 1.99227127e+00
 1.99248076e+00 1.99988197e+00 1.99994098e+00 2.00000000e+00
 2.00005902e+00 2.00751924e+00 2.00757825e+00 2.00772873e+00
 2.00778774e+00 2.01518894e+00 2.01524796e+00 2.01530698e+00
 2.02291767e+00 2.98475204e+00 2.98481106e+00 2.99221226e+00
 2.99227127e+00 2.992421

...

[0.00000000e+00 1.17069807e-08 2.34139614e-08 9.46247733e-06
 9.64002097e-06 1.89249547e-05 1.90907913e-05 1.92800419e-05
 3.81815826e-05 9.74608510e-05 9.98494690e-05 1.06911621e-04
 1.06923328e-04 1.07089165e-04 1.09300239e-04 1.09477783e-04
 1.09489490e-04 1.16551642e-04 1.18928553e-04 1.18940260e-04
 1.94921702e-04 1.99698938e-04 2.13823243e-04 2.13846657e-04
 2.14178330e-04 2.18600479e-04 2.18955566e-04 2.18978980e-04
 2.28429750e-04 2.33079871e-04 2.33103285e-04 2.37857107e-04
 2.37880521e-04 3.94597226e-04 7.79180237e-03 7.87980074e-03
 7.88201182e-03 7.88925151e-03 7.88926322e-03 7.89165184e-03
 7.97001019e-03 1.57596015e-02 1.57640236e-02 1.57785030e-02
 1.57785264e-02 1.57832803e-02 1.57833037e-02 1.59400204e-02
 1.59589219e-02 1.59592770e-02 1.59593004e-02 8.02628203e-02
 8.02628320e-02 8.03530298e-02 8.03530415e-02 8.07061682e-02
 8.07156190e-02 8.07939773e-02 8.08034398e-02 8.08036056e-02
 8.80355436e-02 8.80451719e-02 8.80451953e-02 8.81257531e-02
 8.81353814e-02 8.848833

100%|██████████| 8/8 [00:00<00:00, 31.59it/s]

[0.00000000e+00 2.34139614e-08 1.89249547e-05 1.92800419e-05
 3.81815826e-05 1.94898288e-04 1.99698938e-04 2.14178330e-04
 2.18600479e-04 2.18955566e-04 2.18978980e-04 1.57596015e-02
 1.57640236e-02 1.57785030e-02 1.57832803e-02 1.57833037e-02
 1.59589219e-02 1.60525641e-01 1.60525664e-01 1.61431214e-01
 1.61587955e-01 1.61607235e-01 1.76090344e-01 1.76251506e-01
 1.76976661e-01 1.76995563e-01 1.77152658e-01 3.06353994e-01
 3.06354349e-01 3.06354372e-01 3.06373250e-01 3.06373274e-01
 3.21899440e-01 3.21899795e-01 3.21918697e-01 1.66253586e+00
 1.67790088e+00 1.67808130e+00 1.67810020e+00 1.67810056e+00
 1.69362673e+00 1.69364563e+00 1.82282844e+00 1.82300444e+00
 1.83839277e+00 1.83856876e+00 1.83927468e+00 1.83929392e+00
 1.98404070e+00 1.98404105e+00 1.98405998e+00 1.98421672e+00
 1.98422147e+00 1.98423598e+00 1.98424040e+00 1.99976212e+00
 1.99976214e+00 1.99976692e+00 1.99978102e+00 1.99978140e+00
 1.99978582e+00 1.99978615e+00 1.99980508e+00 1.99996182e+00
 1.99998072e+00 1.999981


100%|██████████| 10/10 [00:00<00:00, 42.65it/s]

['./database/training/patient038/patient038_frame11', './database/training/patient038/patient038_frame11', './database/training/patient038/patient038_frame11', './database/training/patient038/patient038_frame11', './database/training/patient038/patient038_frame11', './database/training/patient038/patient038_frame11', './database/training/patient038/patient038_frame11', './database/training/patient038/patient038_frame11', './database/training/patient038/patient038_frame01', './database/training/patient038/patient038_frame01', './database/training/patient038/patient038_frame01', './database/training/patient038/patient038_frame01', './database/training/patient038/patient038_frame01', './database/training/patient038/patient038_frame01', './database/training/patient038/patient038_frame01', './database/training/patient038/patient038_frame01', './database/training/patient057/patient057_frame01', './database/training/patient057/patient057_frame01', './database/training/patient057/patient057_fr